In [1]:
import requests
import json
import pandas as pd

# Get security-related articles pageviews

In [2]:
ratings = pd.read_csv('data/articles_ratings.csv', index_col=0)
ratings.head()

,Rating
Article,
Datenschutzrecht,4.23
Richtlinie 95/46/EG (Datenschutzrichtlinie),4.08
Datenschutz,4.08
Datenschutzerklärung,4.00
Datenschutz-Grundverordnung,3.92


In [3]:
all_data = []
for article in ratings.index:
    rating = ratings.loc[article, 'Rating']
    url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/{}/monthly/20141001/20171031'.format(article)
    r = requests.get(url)
    if r.status_code == 200:
        data = r.json()
        article_data = data['items']
        for item in article_data:
            item['rating'] = rating
        all_data += article_data
    else:
        print('Problem with article "{}".'.format(article))


with open('data/pageviews.json', 'w') as f:
    json.dump(all_data, f)

Problem with article "Richtlinie 95/46/EG (Datenschutzrichtlinie)".
Problem with article "Google LLC".
Problem with article "Agentur der Europäischen Union für Cybersicherheit".
Problem with article "GAFAM".
Problem with article "Brüssel-Effekt".
Problem with article "Meme (Kulturphänomen)".
Problem with article "Verordnung (EU) Nr. 910/2014 (eIDAS-Verordnung)".
Problem with article "Royal Assent".
Problem with article "Tronc (Unternehmen)".
Problem with article "NOYB".


# Get best article for year 2015 from monthly best

In [4]:
n_best = 50

In [16]:
def get_best_articles(df, year, n_best):

    # Sort by number of views
    df.sort_values(by=['Views'], ascending=False, inplace=True)

    display(df.head(5))

    best = df.iloc[:n_best].index
    print("The {} most viewed articles in {}:".format(n_best, year))
    print(list(best))
    
    return best

def get_best_articles_views(articles, year):

    # mtn j'ai les articles, faut récupérer leur pageviews par mois et les sommer.
    all_data = []
    for article in articles:
        url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/de.wikipedia.org/all-access/user/{}/monthly/{}0101/{}1231'.format(article, year, year)
        r = requests.get(url)
        if r.status_code == 200:
            data = r.json()
            article_data = data['items']
            all_data += article_data
        else:
            print('Problem with article "{}".'.format(article))
    
    with open('data/topviews-{}.json'.format(year), 'w') as f:
        json.dump(all_data, f)

In [17]:
df_2015 = pd.DataFrame(columns=['Page', 'Views'])
for i in range(7, 13):
    df_temp = pd.read_csv('top_data/topviews-2015_{:02d}.csv'.format(i))
    df_temp = df_temp.loc[:,['Page', 'Views']]
    df_2015 = df_2015.append(df_temp, ignore_index=True)

display(df_2015.sample(5))

# sum all month per article
df_2015_sum = df_2015.groupby('Page').sum()

best_2015 = get_best_articles(df_2015_sum, 2015, n_best)
get_best_articles_views(best_2015, 2015)

,Page,Views
1068,Natriumcyanid,108824
5737,Die Tribute von Panem – Catching Fire,51410
5628,Diese Drombuschs,55144
5226,Mutter Teresa,76476
2015,James Bond,139693


,Views
Page,
Template:GeoTemplate,4175778
Deutschland,3978117
Nekrolog 2015,3338582
Syrien,2303624
Islamischer Staat (Organisation),2175951


The 50 most viewed articles in 2015:
['Template:GeoTemplate', 'Deutschland', 'Nekrolog 2015', 'Syrien', 'Islamischer Staat (Organisation)', 'Elli Smula', 'Star Wars', 'James Bond', 'Star Wars: Das Erwachen der Macht', 'Zauberwürfel', 'The Walking Dead (Fernsehserie)', 'Die Tribute von Panem', 'The 100', 'Game of Thrones', 'Helmut Schmidt', 'The Big Bang Theory', 'Helene Fischer', 'Vereinigte Staaten', 'Wikipedia', 'Unix-Shell', 'James Bond 007: Spectre', 'Pablo Escobar', 'Schweiz', 'Berlin', 'Bürgerkrieg in Syrien', 'Angela Merkel', 'Adolf Hitler', 'Österreich', 'Fußball-Europameisterschaft 2016', 'Mitgliedstaaten der Europäischen Union', 'Elyas M’Barek', 'Zweiter Weltkrieg', 'Figuren aus Star Wars', 'Russland', 'Liste der Kfz-Kennzeichen in Deutschland', 'Stefan Raab', 'Asperger-Syndrom', 'Frankreich', 'Halloween', 'Jennifer Lawrence', 'Figuren aus dem Marvel-Universum', 'Europäische Union', 'Two and a Half Men', 'Stephen Hawking', 'Türkei', 'Gutbrod (Unternehmen)', 'Periodensystem', 

In [18]:
df_2016 = pd.DataFrame(columns=['Page', 'Views'])
for i in range(1, 13):
    df_temp = pd.read_csv('top_data/topviews-2016_{:02d}.csv'.format(i))
    df_temp = df_temp.loc[:,['Page', 'Views']]
    df_2016 = df_2016.append(df_temp, ignore_index=True)

display(df_2016.sample(5))

# sum all month per article
df_2016_sum = df_2016.groupby('Page').sum()

best_2016 = get_best_articles(df_2016_sum, 2016, n_best)
get_best_articles_views(best_2016, 2016)

,Page,Views
9196,Dieter Bohlen,63924
6184,Arnold Schwarzenegger,67658
8695,Hans Albers,41946
7759,RAID,42678
2520,Kollegah,63592


,Views
Page,
Deutschland,7106195
Nekrolog 2016,6997010
Donald Trump,5759879
Game of Thrones,4537793
Fußball-Europameisterschaft 2016,4075308


The 50 most viewed articles in 2016:
['Deutschland', 'Nekrolog 2016', 'Donald Trump', 'Game of Thrones', 'Fußball-Europameisterschaft 2016', 'Wikipedia', 'Island', 'Unix-Shell', 'The Walking Dead (Fernsehserie)', 'Vereinigte Staaten', 'Präsidentschaftswahl in den Vereinigten Staaten 2016', 'Bud Spencer', 'Berlin', 'Schweiz', 'Leonardo DiCaprio', 'Olympische Sommerspiele 2016', 'Österreich', 'The Big Bang Theory', 'Cristiano Ronaldo', 'Adolf Hitler', 'Alternative für Deutschland', 'Star Wars', 'Grey’s Anatomy', 'Figuren aus dem Marvel-Universum', 'David Bowie', 'Mitgliedstaaten der Europäischen Union', 'Angela Merkel', 'Pablo Escobar', 'Europäische Union', 'Russland', 'Periodensystem', 'Zweiter Weltkrieg', 'Erster Weltkrieg', 'Vereinigtes Königreich', 'Islamischer Staat (Organisation)', 'Türkei', 'Facebook', 'Liste der Präsidenten der Vereinigten Staaten', 'The Revenant – Der Rückkehrer', 'Asperger-Syndrom', 'Liste der Kfz-Kennzeichen in Deutschland', 'Terence Hill', 'Frankreich', 'Euro

In [21]:
df_2017 = pd.DataFrame(columns=['Page', 'Views'])

for i in range(1, 11):
    df_temp = pd.read_csv('top_data/topviews-2017_{:02d}.csv'.format(i))
    df_temp = df_temp.loc[:,['Page', 'Views']]
    df_2017 = df_2017.append(df_temp, ignore_index=True)

display(df_2017.sample(5))

# sum all month per article
df_2017_sum = df_2017.groupby('Page').sum()

best_2017 = get_best_articles(df_2017_sum, 2017, n_best)
get_best_articles_views(best_2017, 2017)

,Page,Views
5886,Maryam Mirzakhani,99039
8638,Marvel Cinematic Universe,42344
3065,Zinédine Zidane,93073
545,Römisches Reich,70025
9090,Chester Bennington,63875


,Views
Page,
Nekrolog 2017,5649469
Deutschland,5174475
Game of Thrones,3739954
Donald Trump,2906391
Unix-Shell,2449768


The 50 most viewed articles in 2017:
['Nekrolog 2017', 'Deutschland', 'Game of Thrones', 'Donald Trump', 'Unix-Shell', 'Italien', 'Angela Merkel', 'Vereinigte Staaten', 'Wikipedia', 'Martin Luther', 'Berlin', 'Martin Schulz', 'The Walking Dead (Fernsehserie)', 'Österreich', 'Schweiz', 'Alice Weidel', 'Bundestagswahl 2017', 'Prison Break', 'Adolf Hitler', 'Bitcoin', 'Shirin David', 'Frauke Petry', 'Tote Mädchen lügen nicht (Fernsehserie)', 'Russland', 'Liste der Kfz-Kennzeichen in Deutschland', 'Zweiter Weltkrieg', 'Bundespräsident (Deutschland)', 'The Kelly Family', 'Europa', 'Nordkorea', 'Hacker', 'Pretty Little Liars', 'Erster Weltkrieg', 'The Big Bang Theory', 'Chester Bennington', 'Sahra Wagenknecht', 'Helmut Kohl', 'Asperger-Syndrom', 'Helene Fischer', 'Emmanuel Macron', 'Ed Sheeran', 'Grey’s Anatomy', 'Hamburg', 'Alternative für Deutschland', 'Vikings (Fernsehserie)', 'Frankreich', 'Star Wars', 'Katalonien', 'Liste in Deutschland vorhandener Dampflokomotiven', 'Präsenz']
